# A Crash Course in Accelerated Database Recovery

Create a database

Create a table

Insert 1 million rows into the table

This script ran for about 30 minutes on my laptop, so...

In [91]:
USE master;

SET NOCOUNT ON;

DROP DATABASE IF EXISTS AdrDb;

CREATE DATABASE AdrDb;
GO

DROP TABLE IF EXISTS AdrTest;

USE AdrDb;

CREATE TABLE AdrTest
(RowId INT IDENTITY(1,1)
,FirstName NVARCHAR(100)
,LastName NVARCHAR(100));

DECLARE @loopcount INT, 
@looplimit INT;

SELECT @loopcount = 1, @looplimit = 2500;

WHILE @loopcount <= @looplimit
BEGIN

    INSERT INTO AdrTest
    (FirstName
    ,LastName)
    VALUES
    (N'Terence',N'Butler'),
    (N'John',N'Osbourne'),
    (N'Anthony',N'Iommi'),
    (N'William',N'Ward');

    SELECT @loopcount += 1;

END

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:19.965

# Backup Database to Default location

To prevent having to spend 30 minutes loading data, backup the database to the default backup path

For the Linux container I am running on, the default path is <span style="color: rgb(163, 21, 21); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 16px; white-space: pre;">/var/opt/mssql/data/</span>

In [ ]:
USE master
BACKUP DATABASE AdrDb TO DISK=N'/var/opt/mssql/data/AdrDb.bak';  

Processed 384 pages for database 'AdrDb', file 'AdrDb' on file 1.

Processed 2 pages for database 'AdrDb', file 'AdrDb_log' on file 1.

BACKUP DATABASE successfully processed 386 pages in 0.164 seconds (18.364 MB/sec).

Total execution time: 00:00:00.382

# Restore Database

For subsequent runs of the demo, restore the database from backup

In [ ]:
USE master;

ALTER DATABASE AdrDb
SET SINGLE_USER WITH ROLLBACK IMMEDIATE

RESTORE DATABASE AdrDb FROM DISK=N'/var/opt/mssql/data/AdrDb.bak' WITH REPLACE, STATS = 5;

6 percent processed.

10 percent processed.

15 percent processed.

21 percent processed.

25 percent processed.

...

100 percent processed.

Processed 384 pages for database 'AdrDb', file 'AdrDb' on file 1.

Processed 2 pages for database 'AdrDb', file 'AdrDb_log' on file 1.

RESTORE DATABASE successfully processed 386 pages in 0.137 seconds (21.983 MB/sec).

Total execution time: 00:00:01.638

# Check if Accelerated Database Recovery is Enabled

In [ ]:
USE AdrDb;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'AdrDb';

Commands completed successfully.

Total execution time: 00:00:00.053

name,is_accelerated_database_recovery_on
AdrDb,0


# Delete Rows without ADR
This steo performs the following steps:
- Select and store the last log sequence for the database using fn\_dblog
- Delete 2500 rows from the table in an open transaction
- Use fn\_dblog to get a count of the delete operations from the transaction log
- Select and store the last log sequence for the database using fn\_dblog
- Rollback the delete transaction
- Use fn\_dblog to get a count of the compensation records from the transaction log

In [ ]:
USE AdrDb;

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

BEGIN TRANSACTION

DELETE FROM AdrTest
WHERE LastName = 'Osbourne';

SELECT Operation
,[Description]
,AllocUnitName
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE Operation = 'LOP_DELETE_ROWS'
GROUP BY Operation, [Description],AllocUnitName;
GO

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

ROLLBACK TRANSACTION;

SELECT Operation
,[Description]
,AllocUnitName
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE [Description] = 'Compensation'
GROUP BY Operation, [Description],AllocUnitName;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.182

Operation,Description,AllocUnitName,TranCount
LOP_DELETE_ROWS,,dbo.AdrTest,2500


Operation,Description,AllocUnitName,TranCount
LOP_INSERT_ROWS,COMPENSATION,dbo.AdrTest,2500


# Enable Accelerated Database Recovery

In [ ]:
USE master;

ALTER DATABASE AdrDb
SET ACCELERATED_DATABASE_RECOVERY = ON;

SELECT name, is_accelerated_database_recovery_on
FROM sys.databases
WHERE name = N'AdrDb';

Commands completed successfully.

Total execution time: 00:00:00.074

name,is_accelerated_database_recovery_on
AdrDb,1


# Delete Rows with ADR

- Select and store the last log sequence for the database using fn\_dblog
- Delete 250000 rows from the table in an open transaction
- Use fn\_dblog to get a count of the delete operations from the transaction log

In [ ]:
USE AdrDb;

DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

SELECT @maxlsn;

BEGIN TRANSACTION

DELETE FROM AdrTest
WHERE LastName = 'Osbourne';

-- SELECT Operation
-- ,[Description]
-- ,AllocUnitName
-- ,COUNT(*) AS TranCount
-- FROM fn_dblog(@maxlsn,NULL)
-- -- WHERE Operation = 'LOP_INSERT_ROWS'
-- GROUP BY Operation , [Description], AllocUnitName
-- ORDER BY COUNT(*) DESC;

-- SELECT  TOP 1 [Page Id] 
-- FROM fn_dblog(@maxlsn,NULL)
-- WHERE Operation = 'LOP_INSERT_ROWS'
-- AND AllocUnitName = N'sys.persistent_version_store';

SELECT Operation
,[Description]
,AllocUnitName
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE Operation IN ('LOP_DELETE_ROWS', 'LOP_INSERT_ROWS')
GROUP BY Operation, [Description], AllocUnitName;
GO

Commands completed successfully.

Total execution time: 00:00:00.171

(No column name)
0x00000029:000017b0:0001


Operation,Description,AllocUnitName,TranCount
LOP_DELETE_ROWS,,dbo.AdrTest,2500
LOP_INSERT_ROWS,,sys.persistent_version_store,1100


# Rollback Transaction with ADR

- Get the last lsn from the transaction log using fn\_dblog
- Rollback the delete transaction
- Get all log records after the delete
- Check for compensation records

In [89]:
DECLARE @maxlsn NVARCHAR(46);

SELECT @maxlsn = CONCAT(N'0x',MAX([Current LSN])) FROM fn_dblog(NULL,NULL);

ROLLBACK TRANSACTION;

SELECT [Current LSN]
,[Transaction ID]
,[Transaction Name]
,Operation
,Context
,[Description]
,[Previous LSN]
,AllocUnitName
,[Page ID]
,[Slot ID]
,[Begin Time]
,[Database Name]
,[Number of Locks]
,[Lock Information]
,[New Split Page]
FROM fn_dblog(@maxlsn,NULL);

SELECT Operation
,[Description]
,COUNT(*) AS TranCount
FROM fn_dblog(@maxlsn,NULL)
WHERE [Description] = 'Compensation'
GROUP BY Operation, [Description];
GO

Commands completed successfully.

Total execution time: 00:00:00.199

Current LSN,Transaction ID,Transaction Name,Operation,Context,Description,Previous LSN,AllocUnitName,Page ID,Slot ID,Begin Time,Database Name,Number of Locks,Lock Information,New Split Page
00000029:00002068:006e,0000:00000d5e,NULL,LOP_COMMIT_XACT,LCX_NULL,2021/03/02 20:15:43:247,00000029:00001f78:0002,NULL,NULL,NULL,NULL,none,NULL,NULL,NULL
00000029:00002068:006f,0000:00000d56,NULL,LOP_ABORT_XACT,LCX_CTR_ABORTED,2021/03/02 20:18:21:043,00000029:000017c0:0001,NULL,NULL,NULL,NULL,none,NULL,NULL,NULL


Operation,Description,TranCount
